# Small Http service to get datetime

We will build a small HTTP REST service to gradually build up an understanding of several concepts

- All the different ways you can declare parameters to a python function
- The different python scopes
- What namespaces are

We will gradually add more functions to the little service including a small "database" which will be a set of local
files that we can do simple CRUD operations on and some analytics.

In [ ]:
from typing import Literal, TypeAlias
from fastapi import FastAPI, HTTPException
from datetime import datetime, timedelta, timezone
from pydantic import BaseModel

app = FastAPI()

TimeUnits: TypeAlias = Literal["days", "hours", "minutes", "seconds"]
TimeDelta: TypeAlias = dict[TimeUnits, float]


class Interval(BaseModel):
    from_delta: TimeDelta = {}
    until_delta: TimeDelta = {}

@app.get("/dt/time")
def date_now():
    return {"time": datetime.now().isoformat()}

@app.get("/dt/time_from")
def date_from(days_back: int):
    now = datetime.now(tz=timezone.utc)
    start = now - timedelta(days=days_back)
    return {"time": start.isoformat()}

@app.post("/dt/interval")
def get_interval(interval: Interval):
    now = datetime.now(tz=timezone.utc)
    start = now - timedelta(**interval.from_delta)
    end = now - timedelta(**interval.until_delta)
    if start > end:
        raise HTTPException(status_code=400, detail="from_delta must be greater than until_delta")
    return {"interval": f"{start.isoformat()}/{end.isoformat()}"}

import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()